# Setup

In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import random
import matplotlib.colors as mcolors
from ipywidgets import interact, widgets, interactive, fixed, interact_manual
import random
import matplotlib.colors as mcolors
import time
import colorama
import pickle

with open("Teach_linear_regression.pkl", "rb") as f:
    (
        x, y, COEF_A, COEF_B,
        COEF_A_MIN, COEF_A_MAX,
        COEF_B_MIN, COEF_B_MAX,
        WORST_A_PARAM, WORST_B_PARAM,
        MIN_COST, MAX_COST,
        MIN_VALUE, MAX_VALUE, NB_ELEMENTS,
    ) = pickle.load(f)

In [69]:
def plot_function_with_derivative(f, x_point=1, h=0.5):
    # Generate points for plotting
    MINIMUM_Y = -2
    x = np.linspace(-1, 3, 1000)
    y = f(x)

    # Create the plot
    plt.figure(figsize=(12, 6))

    # Plot main function
    plt.plot(x, y, 'b-', label='f(x)')

    # Calculate points for derivative visualization
    x_plus_h = x_point + h
    x_minus_h = x_point - h
    y_point = f(x_point)
    y_plus_h = f(x_plus_h)
    y_minus_h = f(x_minus_h)

    # Plot points
    plt.plot(x_point, y_point, 'X', color='red', label=f'f({x_point:.2f})', markersize=10)
    plt.plot(x_point + h, y_plus_h, '+', color='gray', alpha=0.5, markersize=8)
    plt.plot(x_point - h, y_minus_h, '+', color='gray', alpha=0.5, markersize=8)

    # Plot secant line through the two points
    x_line = np.array([x_point - h, x_point + h])
    y_line = np.array([y_minus_h, y_plus_h])
    plt.plot(x_line, y_line, "-", color="gray", linewidth=2)

    # bottom_horizontal_line_x = np.array([x_minus_h, x_plus_h])
    # bottom_horizontal_line_y = np.array([y_minus_h, y_minus_h])

    top_horizontal_line_x = np.array([x_minus_h, x_plus_h])
    top_horizontal_line_y = np.array([y_plus_h, y_plus_h])

    left_vertical_line_x = np.array([x_minus_h, x_minus_h])
    left_vertical_line_y = np.array([y_minus_h, y_plus_h])

    h_minus_gray_line_x = np.array([x_minus_h, x_minus_h])
    h_minus_gray_line_y = np.array([-10, y_minus_h])

    h_plus_gray_line_x = np.array([x_plus_h, x_plus_h])
    h_plus_gray_line_y = np.array([-10, y_plus_h])

    # right_vertical_line_x = np.array([x_plus_h, x_plus_h])
    # right_vertical_line_y = np.array([y_minus_h, y_plus_h])
    # Add text annotations for dy and dx with their values
    dy = y_plus_h - y_minus_h
    dx = x_plus_h - x_minus_h
    derivative = dy/dx

    # Using matplotlib's text with math mode and custom colors
    plt.text(-0.85, 3.7, r"$\mathbf{dy} = " + f"{dy:.2f}$", color='orange', fontsize=12)
    plt.text(-0.85, 3.3, r"$\mathbf{dx} = " + r"= 2 \times h " + f" = {dx:.2f} " + "$", color='purple', fontsize=12)

    # Add the ratio calculation with colored values and result
    plt.text(-0.85, 2.6, 
             r"$\mathbf{" +
             r"\frac{dy}{dx} = " + 
             r"\frac{" + f"{dy:.2f}" + "}{" + f"{dx:.2f}" + "}} = " + 
             f"{derivative:.2f}$",
             color='gray', fontsize=14)
    # if y_minus_h > y_plus_h:
    # plt.plot(bottom_horizontal_line_x, bottom_horizontal_line_y, ":", color="purple", linewidth=2, label="Horizontal line")
    # plt.plot(right_vertical_line_x, right_vertical_line_y, ":", color="orange", linewidth=2, label="Vertical line")
    # else:
    plt.plot(top_horizontal_line_x, top_horizontal_line_y, "-", color="purple", linewidth=2, label="$dx = 2 h$")
    plt.plot(left_vertical_line_x, left_vertical_line_y, "-", color="orange", linewidth=2, label=r"$dy = \frac{f(x+h) - f(x-h)}{2h}$")
    plt.plot(h_minus_gray_line_x, h_minus_gray_line_y, ":", color="gray", linewidth=2, label="x-h")
    plt.plot(h_plus_gray_line_x, h_plus_gray_line_y, ":", color="gray", linewidth=2, label="x+h")

    plt.text(x_plus_h + 0.05, MINIMUM_Y +0.5, r"x+h", color="gray", fontsize=12)
    plt.text(x_minus_h - 0.15, MINIMUM_Y + 0.5, r"x-h", color="gray", fontsize=12)
    # Extend the line in both directions
    slope = (y_plus_h - y_minus_h) / (2*h)
    x_extended = np.array([x_point - 5, x_point + 5])
    y_extended = slope * (x_extended - x_point) + y_point
    # Plot both the segment between points and extended line
    # plt.plot(x_extended, y_extended, '--', color='gray', label='Extended secant')
    plt.grid(True)
    plt.legend()
    plt.title(f'Function with Derivative Approximation (h={h:.2f})')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.xlim(-1, 3)
    plt.ylim(-2, 4)

def chaotic_function(x):
    return np.sin(5*x**2) * np.cos(x/2) + np.exp(-((x-3)**2)/10) * np.sin(5*x) + 0.5 * np.tanh(x/2)



# Teach

## Goal

Know how we should update theta to reduce loss ?


This is : $ \frac{d \text{loss}}{d a} $ & $\frac{d \text{loss}}{d b} $


But, what's the derivate ?

In [70]:
@interact(
    x_point=widgets.FloatSlider(
        value=1,
        min=0.0,
        max=2.0,
        step=0.01,
        description="X:",
    ),
    h=widgets.FloatSlider(
        value=0.15,
        min=0.01,
        max=0.5,
        step=0.01,
        description="h:",
    ),
)
def interactive_derivative(x_point, h):
    # f = lambda x: x**2
    f = chaotic_function
    plot_function_with_derivative(f, x_point, h)

interactive(children=(FloatSlider(value=1.0, description='X:', max=2.0, step=0.01), FloatSlider(value=0.15, de…

## Derivate intuition
  - We can use limits to compute an analytical derivate

### $\frac{df(x)}{dx} = \lim_{h \to 0} \frac{f(x+h) - f(x-h)}{2h}$

 - Cheat sheet
   - Direct value
     - Constant: 
       - $\frac{d}{dx}[c] = 0$
     - Add: 
       - $\frac{d}{dx}[f(x) + g(x)] = \frac{d}{dx}f(x) + \frac{d}{dx}g(x)$
     - Multiply: 
       - $\frac{d}{dx}[c \cdot f(x)] = c \cdot \frac{d}{dx}f(x)$
     - Power: 
       - $\frac{d}{dx}[x^n] = n \cdot x^{n-1}$
   - Chain rule: 
     - $\frac{d}{dx}[f(g(x))] = f'(g(x)) \cdot g'(x) = \frac{df(x)}{dg(x)} \frac{dg(x)}{dx}$

![Derivatives](./data/derivatives_cheat_sheet.png)

## Let's apply it to our Linear Regression


2 - Let's apply it to our graph
  - High level view of our graph of computation
![](./data/derivative_draw_0_high_level.jpg)
    - Goal: 
      - d loss / d theta
      - Chain rule:
        -   d loss / d LinReg
        -   d LinReg / d theta
  - Let's go further
    - Show :
      - Detailed computations
![](./data/derivative_draw_1_low_level.jpg)
      - Full chain rule
![](./data/derivative_draw_1.5_chain_rule.jpg)
    - The path we will take :
![](./data/derivative_draw_2_high_level_path.jpg)
      - For each step: d out / d in 
      - We will need to multiply everything after
    - Let's go : 
      - Obtain the derivatives 1 by 1
![](./data/derivative_draw_4_what_to_compute.jpg)
      - Multiply everything
![](./data/derivative_draw_5_complete.jpg)
      - Done !

In [7]:
# ------------------------------ Base functions ------------------------------ #

def predict(x, thetas):
    a, b = thetas
    y_hat = (a * x) + b
    return y_hat


def get_cost(y, y_hat):
    return ((y_hat - y) ** 2).mean()


# -------------------------------- Derivation -------------------------------- #


def get_d_cost_by_d_a(y, y_hat, a):
    return 2 * (y_hat - y) * x

def get_d_cost_by_d_b(y, y_hat, b):
    return 2 * (y_hat - y)

def get_parameter_derivative(x, y, a, b):
    y_hat = predict(x, (a, b))

    d_cost_by_d_a = get_d_cost_by_d_a(y, y_hat, a).mean()
    d_cost_by_d_b = get_d_cost_by_d_b(y, y_hat, b).mean()

    return d_cost_by_d_a, d_cost_by_d_b


# ----------------------------- Finite difference ---------------------------- #


def get_parameter_finite_difference(x, y, a, b):
    epsilon = 0.5
    base_cost = get_cost(y, predict(x, (a, b)))
    a_minus_epsilon = a - epsilon
    b_minus_epsilon = b - epsilon

    cost_a_minus_epsilon = get_cost(y, predict(x, (a_minus_epsilon, b)))
    cost_b_minus_epsilon = get_cost(y, predict(x, (a, b_minus_epsilon)))
    # print(f"{base_cost = } {cost_a_epsilon = } {cost_b_epsilon = }")

    cost_a_finite_difference = (base_cost - cost_a_minus_epsilon) / epsilon
    cost_b_finite_difference = (base_cost - cost_b_minus_epsilon) / epsilon
    # print(f"{cost_a_finite_difference = } {cost_b_finite_difference = }")

    return cost_a_finite_difference, cost_b_finite_difference


def get_parameter_finite_difference_precision(x, y, a, b):
    epsilon = 0.5
    # base_cost = get_cost(y, predict(x, (a, b)))
    a_plus_epsilon = a + epsilon
    b_plus_epsilon = b + epsilon

    a_minus_epsilon = a - epsilon
    b_minus_epsilon = b - epsilon

    cost_a_plus_epsilon = get_cost(y, predict(x, (a_plus_epsilon, b)))
    cost_b_plus_epsilon = get_cost(y, predict(x, (a, b_plus_epsilon)))
    cost_a_minus_epsilon = get_cost(y, predict(x, (a_minus_epsilon, b)))
    cost_b_minus_epsilon = get_cost(y, predict(x, (a, b_minus_epsilon)))
    # print(f"{base_cost = } {cost_a_plus_epsilon = } {cost_b_plus_epsilon = } {cost_a_minus_epsilon = } {cost_b_minus_epsilon = }")

    cost_a_finite_difference = (cost_a_plus_epsilon - cost_a_minus_epsilon) / (2 * epsilon)
    cost_b_finite_difference = (cost_b_plus_epsilon - cost_b_minus_epsilon) / (2 * epsilon)
    # print(f"{cost_a_finite_difference = } {cost_b_finite_difference = }")

    return cost_a_finite_difference, cost_b_finite_difference


# --------------------------------- Optimize --------------------------------- #


def optimize_parameter(x, y, a, b, learning_rate, nb_iterations, derivative_method):
    print(
        f"\nOptimizing parameter with {colorama.Fore.GREEN}{derivative_method}{colorama.Fore.RESET}"
    )
    start_time = time.time()
    for _ in range(nb_iterations):
        if derivative_method == "derivative":
            d_cost_by_d_a, d_cost_by_d_b = get_parameter_derivative(x, y, a, b)
        elif derivative_method == "finite_difference":
            d_cost_by_d_a, d_cost_by_d_b = get_parameter_finite_difference(
                x, y, a, b
            )
        elif derivative_method == "finite_difference_precision":
            d_cost_by_d_a, d_cost_by_d_b = get_parameter_finite_difference_precision(
                x, y, a, b
            )
        a = a - (learning_rate * d_cost_by_d_a)
        b = b - (learning_rate * d_cost_by_d_b)
    end_time = time.time()
    cost = get_cost(y, predict(x, (a, b)))
    print(
        f"Time taken: {colorama.Fore.YELLOW}{end_time - start_time} seconds{colorama.Fore.RESET} \n"
        f"\twith {derivative_method}\n"
        f"\t{a = }\n"
        f"\t{b = }\n"
        f"\t{colorama.Fore.RED}{cost = }{colorama.Fore.RESET}\n"
    )
    return a, b


print(f"IDEAL: {COEF_A = }")
print(f"IDEAL: {COEF_B = }")
learning_rate = 1e-4
nb_iterations = 100_000


a, b = optimize_parameter(
    x, y, WORST_A_PARAM, WORST_B_PARAM, learning_rate, nb_iterations, "finite_difference"
)
# a, b = optimize_parameter(
#     x, y, WORST_A_PARAM, WORST_B_PARAM, learning_rate, nb_iterations, "finite_difference_precision"
# )
a, b = optimize_parameter(
    x, y, WORST_A_PARAM, WORST_B_PARAM, learning_rate, nb_iterations, "derivative"
)

IDEAL: COEF_A = 2.7684667311238043
IDEAL: COEF_B = 3.073489251946131

Optimizing parameter with finite_difference
Time taken: 2.5448598861694336 seconds 
	with finite_difference
	a = 2.245177823764299
	b = 3.588318773979193
	cost = 0.45820155384552125


Optimizing parameter with derivative
Time taken: 1.6806817054748535 seconds 
	with derivative
	a = 2.578090939614186
	b = 3.177332679174755
	cost = 0.371116406168403



IDEAL: COEF_A = 2.544502850465683
IDEAL: COEF_B = 2.39629117885467
Time taken: 0.021463394165039062 seconds with derivative = True
DERIVATIVE: a = 1.8909630721574668e+176
DERIVATIVE: b = 3.781926144314933e+176
Time taken: 0.021030664443969727 seconds with derivative = False
FINITE DIFFERENCE: a = nan
FINITE DIFFERENCE: b = nan


/tmp/ipykernel_1247998/1646815893.py:10: RuntimeWarning: overflow encountered in square
  return ((y - y_hat) ** 2).sum()
/tmp/ipykernel_1247998/1646815893.py:39: RuntimeWarning: invalid value encountered in scalar subtract
  cost_a_finite_difference = (cost_a_epsilon - base_cost)
/tmp/ipykernel_1247998/1646815893.py:40: RuntimeWarning: invalid value encountered in scalar subtract
  cost_b_finite_difference = (cost_b_epsilon - base_cost)
